In [2]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Polygon
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
%matplotlib inline

## 雙北

In [5]:
park_df = pd.read_csv('停車/新北市停車場/新北市路外公共停車場資訊_0001031672146283909812.csv', encoding='utf-8')

In [6]:
park_df.head(2)

,ID,AREA,NAME,TYPE,SUMMARY,ADDRESS,TEL,PAYEX,SERVICETIME,TW97X,TW97Y,TOTALCAR,TOTALMOTOR,TOTALBIKE
0,10056,板橋區,遠東百貨停車場,2,立體式建築附設停車空間,板橋區中山路一段152號,NaN,小型車計時60元;,0~24時,296882.0,2767068.0,453,0,NaN
1,30034,中和區,南華停車場,2,平面式臨時路外停車場,中和區中和路281號邊,NaN,小型車計時20元;小型車月租3500元;,6~24時,301086.0,2765825.0,42,0,NaN


In [77]:
park_df = pd.read_csv('停車/新北市停車場/新北市路外公共停車場資訊_0001031672146283909812.csv', encoding='utf-8')
geometry = [Point(xy) for xy in zip(park_df['TW97X'].values, park_df['TW97Y'].values)]
park_df = gpd.GeoDataFrame(park_df, crs="epsg:3826", geometry=geometry)
park_df = park_df.to_crs("epsg:3824")
park_df.crs = {'init': 'epsg:3824'}
park_df['lat'] = park_df.geometry.y
park_df['lng'] = park_df.geometry.x
park_df = park_df.drop('geometry', axis=1)

In [78]:
park_df.head(2)

,ID,AREA,NAME,TYPE,SUMMARY,ADDRESS,TEL,PAYEX,SERVICETIME,TW97X,TW97Y,TOTALCAR,TOTALMOTOR,TOTALBIKE,lat,lng
0,10056,板橋區,遠東百貨停車場,2,立體式建築附設停車空間,板橋區中山路一段152號,NaN,小型車計時60元;,0~24時,296882.0,2767068.0,453,0,NaN,25.010925,121.464492
1,30034,中和區,南華停車場,2,平面式臨時路外停車場,中和區中和路281號邊,NaN,小型車計時20元;小型車月租3500元;,6~24時,301086.0,2765825.0,42,0,NaN,24.999567,121.506097


In [89]:
park_df = park_df.rename(columns={'TOTALCAR':'CAR_NUM', 'TOTALMOTOR':'MOTOR_NUM'})
park_df = park_df[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]
park_df.head(2)

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,遠東百貨停車場,板橋區中山路一段152號,453,0,25.010925,121.464492
1,南華停車場,中和區中和路281號邊,42,0,24.999567,121.506097


In [90]:
park_df.to_csv('停車/新北市停車場/新北停車場(processed).csv', index=False, encoding='utf-8-sig')

In [99]:
park_df = gpd.read_file('停車/台北市停車場/park05.shp', encoding='utf-8')
park_df = park_df.drop(['GEOM', 'GEOM4326', 'NUM_MH', 'ZONE1'], axis=1)
park_df.crs = {'init': 'epsg:3826'}
park_df = park_df.to_crs("epsg:3824")
park_df.crs = {'init': 'epsg:3824'}
park_df['lat'] = park_df.geometry.centroid.y
park_df['lng'] = park_df.geometry.centroid.x
park_df = park_df.drop('geometry', axis=1)
park_df.loc[:, ['NUM_S', 'NUM_M', 'NUM_L']] = park_df.loc[:, ['NUM_S', 'NUM_M', 'NUM_L']].fillna(0)
park_df.loc[:, ['NUM_S', 'NUM_M', 'NUM_L']] = park_df.loc[:, ['NUM_S', 'NUM_M', 'NUM_L']].astype('int')
park_df['NUM_S'] = park_df['NUM_S'] + park_df['NUM_L']
park_df = park_df.rename(columns={'GATENAME':'NAME', 'GATEADRS':'ADDRESS','NUM_S':'CAR_NUM', 'NUM_M':'MOTOR_NUM'})
park_df = park_df[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

C:\Users\BH\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [100]:
park_df['CAR_NUM'].sum()

185163

In [94]:
park_df.head(2)

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,臺北市立金華國民中學,新生南路2段32號,49,0,25.031408,121.533080
1,墾海停車場,忠孝東路1段23號B2、B3,66,0,25.044875,121.524818


In [101]:
park_df.to_csv('停車/台北市停車場/台北停車場(processed).csv', index=False, encoding='utf-8-sig')

## 新竹

In [106]:
park_df = pd.read_csv('停車/新竹停車場/新竹縣停車場.csv', encoding='utf-8')

In [120]:
park_df

,證號,公司名稱,公司電話,停車場名稱,行政區,停車場地址-地號,型式,停車格數量,收費方式,備註
0,1,福松建設股份有限公司,03-5535388,竹北市縣治停車場,竹北市,竹北市縣華段142地號(竹北市光明六路89號),立體,小型車758格,計時、月租,NaN
1,003-3,湖口鄉公所,03-5993911-292,湖口鄉中興立體停車場,湖口鄉,湖口鄉興湖段58-74等17筆地號,立體,小型車147格,計時、月租,NaN
2,007-8,國雲公寓大廈管理維護股份有限公司,02-25573722,竹東仁愛路公有停車場(廣停二),竹東鎮,新竹縣竹東鎮仁愛路/學前路口(竹東鎮仁愛段1252、1254、1255、1256、1257、...,平面,小型車163格,計時、月租,NaN
3,008-7,國雲公寓大廈管理維護股份有限公司,02-25573722,華興街立體停車場,竹北市,竹北市華興段129、132、133、134、197、236、237、239地號(竹北市竹義里...,立體,小型車135格,計時、月租,NaN
4,010-7,國雲公寓大廈管理維護股份有限公司,02-25573722,中正東路停車場,竹北市,竹北市竹仁段1024地號,平面,小型車45格,計時、月租,NaN
...,...,...,...,...,...,...,...,...,...,...
63,92,?亭停車事業股份有限公司,02-27888750-8813,道禾二平面停車場,竹北市,竹北市自強段167、167-1、167-2地號,平面,小型車41格,計時、月租,NaN
64,93,巨人社區管理委員會,03-6580380,巨人社區停車場,竹北市,竹北市莊敬段269地號,平面,小型車62格,計時、月租,NaN
65,94,大榆二生活建築有限公司,936321599,打里摺．大榆二社區附設停車場,竹北市,竹北市永興段481地號,平面,小型車35格,計時,NaN
66,95,陳信源,02-27888750-8813黃小姐,竹北新國停車場,竹北市,竹北市新國段824、825-1地號,平面,小型車19格,計時、月租,NaN


In [127]:
park_df['CAR_NUM'] = 0
park_df['MOTOR_NUM'] = 0
for i, l in enumerate(park_df.停車格數量.str.split('、')):
    for s in l:
        print(i, s, ''.join([x for x in s if x.isdigit()]))
        if s[:3] == '小型車':
            park_df.loc[i, 'CAR_NUM'] += int(''.join([x for x in s if x.isdigit()]))
        elif s[:3] == '大型車':
            park_df.loc[i, 'CAR_NUM'] += int(''.join([x for x in s if x.isdigit()]))
        elif s[:2] == '機車':
            park_df.loc[i, 'MOTOR_NUM'] += int(''.join([x for x in s if x.isdigit()]))

0 小型車758格 758
1 小型車147格 147
2 小型車163格 163
3 小型車135格 135
4 小型車45格 45
5 小型車103格 103
6 小型車68格 68
7 小型車799格 799
7 機車917格 917
8 小型車66格 66
8 機車190格 190
9 小型車68格 68
10 小型車48格 48
11 小型車37格 37
12 小型車174格 174
13 小型車309格 309
14 小型車36格 36
15 小型車109格 109
16 小型車843格 843
17 小型車119格 119
17 機車34格 34
18 小型車145格 145
19 小型車475格 475
20 小型車326格 326
21 小型車47格 47
22 小型車150格 150
23 小型車28格 28
24 小型車9格 9
25 小型車43格 43
26 小型車53格 53
27 小型車99格 99
28 小型車78格 78
29 小型車223格 223
30 小型車11格 11
31 小型車19格 19
32 小型車27格 27
33 小型車90格 90
34 小型車78格 78
35 小型車58格 58
36 小型車54格 54
37 小型車46格 46
38 小型車14格 14
39 小型車18格 18
40 小型車38格 38
41 小型車28格 28
42 小型車116格 116
43 小型車57格 57
44 小型車36格 36
45 小型車72格 72
46 小型車127格 127
47 小型車48格 48
48 小型車63格 63
49 大型車(休旅車)24格 24
49 小型車56格 56
50 小型車20格 20
51 小型車556格 556
51 機車500格 500
52 小型車67格 67
53 小型車68格 68
54 小型車128格 128
55 小型車64格 64
56 機車373格 373
57 小型車532格 532
57 機車150格 150
58 小型車32格 32
59 大客車92格 92
59 小型車2275格 2275
59 機車910格 910
60 小型車21格 21
61 小型車39格 39
62 小型車154格 154
63 小型車41格 41
64 小型車62格 62
65 小型車3

In [128]:
park_df

,證號,公司名稱,公司電話,停車場名稱,行政區,停車場地址-地號,型式,停車格數量,收費方式,備註,CAR_NUM,MOTOR_NUM
0,1,福松建設股份有限公司,03-5535388,竹北市縣治停車場,竹北市,竹北市縣華段142地號(竹北市光明六路89號),立體,小型車758格,計時、月租,NaN,758,0
1,003-3,湖口鄉公所,03-5993911-292,湖口鄉中興立體停車場,湖口鄉,湖口鄉興湖段58-74等17筆地號,立體,小型車147格,計時、月租,NaN,147,0
2,007-8,國雲公寓大廈管理維護股份有限公司,02-25573722,竹東仁愛路公有停車場(廣停二),竹東鎮,新竹縣竹東鎮仁愛路/學前路口(竹東鎮仁愛段1252、1254、1255、1256、1257、...,平面,小型車163格,計時、月租,NaN,163,0
3,008-7,國雲公寓大廈管理維護股份有限公司,02-25573722,華興街立體停車場,竹北市,竹北市華興段129、132、133、134、197、236、237、239地號(竹北市竹義里...,立體,小型車135格,計時、月租,NaN,135,0
4,010-7,國雲公寓大廈管理維護股份有限公司,02-25573722,中正東路停車場,竹北市,竹北市竹仁段1024地號,平面,小型車45格,計時、月租,NaN,45,0
...,...,...,...,...,...,...,...,...,...,...,...,...
63,92,?亭停車事業股份有限公司,02-27888750-8813,道禾二平面停車場,竹北市,竹北市自強段167、167-1、167-2地號,平面,小型車41格,計時、月租,NaN,41,0
64,93,巨人社區管理委員會,03-6580380,巨人社區停車場,竹北市,竹北市莊敬段269地號,平面,小型車62格,計時、月租,NaN,62,0
65,94,大榆二生活建築有限公司,936321599,打里摺．大榆二社區附設停車場,竹北市,竹北市永興段481地號,平面,小型車35格,計時,NaN,35,0
66,95,陳信源,02-27888750-8813黃小姐,竹北新國停車場,竹北市,竹北市新國段824、825-1地號,平面,小型車19格,計時、月租,NaN,19,0


In [129]:
park_df.to_csv('停車/新竹停車場/新竹縣停車場_.csv', encoding='utf-8')

In [139]:
park_df = pd.read_csv('停車/新竹停車場/新竹市停車場_.csv', encoding='ansi')
park_df = park_df.drop(['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'],axis=1)

In [143]:
park_df.loc[:, '汽車 車位數'] = park_df['汽車 車位數'].fillna(0)
park_df.loc[:, '機車 車位數'] = park_df['機車 車位數'].fillna(0)

In [144]:
park_df

,停車場名稱,汽車 車位數,機車 車位數,停車場地址,營運時間,停車費用-平日,停車費用-假日,計費基準,補充說明,月租,備註說明,lat,lng
0,南門機械停車場,150.0,0.0,新竹市中華路二段545之1號,24小時營運,20元,20元,計時,委外,開放月租,24小時營運 (委外),24.499926,120.968500
1,民富機械停車場,240.0,0.0,新竹市少年街49巷10號,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.806644,120.955865
2,府後地下停車場,296.0,0.0,新竹市府後街42號,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.807405,120.969843
3,東大陸橋停車場-中央站,216.0,0.0,新竹市東大路橋下,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.803374,120.967693
4,東大陸橋停車場-中華站,146.0,0.0,新竹市東大路橋下,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.806089,120.973653
5,演藝廳廣場停車場,107.0,0.0,演藝路與東大路口,8～22時,20元,20元,計時,平日與例假日收費,開放月租,8～22時 (平日與例假日收費),24.812441,120.967993
6,明志書院停車場,610.0,800.0,新竹市西大路317號,24小時營運,30元,30元,計時,委外,開放月租,24小時營運 (委外),24.801574,120.965691
7,赤土崎地下停車場,602.0,210.0,新竹市建中路2號,24小時營運,20元,20元,計時,委外,開放月租,24小時營運 (委外),24.799265,120.993970
8,停七停車場,106.0,0.0,新竹市西大路41號,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運,24.795221,120.971814
9,振興陸橋停車場,85.0,45.0,新竹市振興路橋下,8～22時,20元,20元,計時,平日與例假日收費,開放月租,8～22時 (平日與例假日收費),24.795846,120.964994


In [146]:
park_df

,停車場名稱,CAR_NUM,MOTOR_NUM,停車場地址,營運時間,停車費用-平日,停車費用-假日,計費基準,補充說明,月租,備註說明,lat,lng
0,南門機械停車場,150.0,0.0,新竹市中華路二段545之1號,24小時營運,20元,20元,計時,委外,開放月租,24小時營運 (委外),24.499926,120.968500
1,民富機械停車場,240.0,0.0,新竹市少年街49巷10號,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.806644,120.955865
2,府後地下停車場,296.0,0.0,新竹市府後街42號,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.807405,120.969843
3,東大陸橋停車場-中央站,216.0,0.0,新竹市東大路橋下,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.803374,120.967693
4,東大陸橋停車場-中華站,146.0,0.0,新竹市東大路橋下,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運 (委外),24.806089,120.973653
5,演藝廳廣場停車場,107.0,0.0,演藝路與東大路口,8～22時,20元,20元,計時,平日與例假日收費,開放月租,8～22時 (平日與例假日收費),24.812441,120.967993
6,明志書院停車場,610.0,800.0,新竹市西大路317號,24小時營運,30元,30元,計時,委外,開放月租,24小時營運 (委外),24.801574,120.965691
7,赤土崎地下停車場,602.0,210.0,新竹市建中路2號,24小時營運,20元,20元,計時,委外,開放月租,24小時營運 (委外),24.799265,120.993970
8,停七停車場,106.0,0.0,新竹市西大路41號,24小時營運,20元,20元,計時,NaN,開放月租,24小時營運,24.795221,120.971814
9,振興陸橋停車場,85.0,45.0,新竹市振興路橋下,8～22時,20元,20元,計時,平日與例假日收費,開放月租,8～22時 (平日與例假日收費),24.795846,120.964994


In [149]:
park_df = park_df.rename(columns={'停車場名稱 ':'NAME', '停車場地址 ':'ADDRESS','汽車 車位數':'CAR_NUM', '機車 車位數':'MOTOR_NUM'})
park_df = park_df[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [151]:
park_df.to_csv('停車/新竹停車場/新竹市停車場(processed).csv', index=False, encoding='utf-8-sig')

In [156]:
park_df = pd.read_csv('停車/新竹停車場/新竹縣停車場_.csv', encoding='utf-8')

In [159]:
park_df['停車場地址-地號'] = park_df['停車場地址-地號'].str.split('、').str[0] + '號'
park_df['停車場地址-地號'] = park_df['停車場地址-地號'].str.split('地號').str[0] + '號'
park_df['停車場地址-地號'] = park_df['停車場地址-地號'].str.split('號').str[0] + '號'

In [160]:
park_df

,Unnamed: 0,證號,公司名稱,公司電話,停車場名稱,行政區,停車場地址-地號,型式,停車格數量,收費方式,備註,CAR_NUM,MOTOR_NUM
0,0,1,福松建設股份有限公司,03-5535388,竹北市縣治停車場,竹北市,竹北市縣華段142號,立體,小型車758格,計時、月租,NaN,758,0
1,1,003-3,湖口鄉公所,03-5993911-292,湖口鄉中興立體停車場,湖口鄉,湖口鄉興湖段58-74等17筆號,立體,小型車147格,計時、月租,NaN,147,0
2,2,007-8,國雲公寓大廈管理維護股份有限公司,02-25573722,竹東仁愛路公有停車場(廣停二),竹東鎮,新竹縣竹東鎮仁愛路/學前路口(竹東鎮仁愛段1252號,平面,小型車163格,計時、月租,NaN,163,0
3,3,008-7,國雲公寓大廈管理維護股份有限公司,02-25573722,華興街立體停車場,竹北市,竹北市華興段129號,立體,小型車135格,計時、月租,NaN,135,0
4,4,010-7,國雲公寓大廈管理維護股份有限公司,02-25573722,中正東路停車場,竹北市,竹北市竹仁段1024號,平面,小型車45格,計時、月租,NaN,45,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,92,?亭停車事業股份有限公司,02-27888750-8813,道禾二平面停車場,竹北市,竹北市自強段167號,平面,小型車41格,計時、月租,NaN,41,0
64,64,93,巨人社區管理委員會,03-6580380,巨人社區停車場,竹北市,竹北市莊敬段269號,平面,小型車62格,計時、月租,NaN,62,0
65,65,94,大榆二生活建築有限公司,936321599,打里摺．大榆二社區附設停車場,竹北市,竹北市永興段481號,平面,小型車35格,計時,NaN,35,0
66,66,95,陳信源,02-27888750-8813黃小姐,竹北新國停車場,竹北市,竹北市新國段824號,平面,小型車19格,計時、月租,NaN,19,0


In [163]:
import googlemaps

gmaps = googlemaps.Client(key='')

In [166]:
for i, address in enumerate(park_df['停車場地址-地號'].values): 
    index = i
    if (i+1)%50 == 0:
        print('Already process %d geocoding requests.'%(i+1))
    geocode_result = gmaps.geocode(address, region='tw')
    try:
        park_df.loc[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
        park_df.loc[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']
    except:
        print('*error at index %d.'%(i))

*error at index 48.
Already process 50 geocoding requests.


In [167]:
park_df

,Unnamed: 0,證號,公司名稱,公司電話,停車場名稱,行政區,停車場地址-地號,型式,停車格數量,收費方式,備註,CAR_NUM,MOTOR_NUM,lat,lng
0,0,1,福松建設股份有限公司,03-5535388,竹北市縣治停車場,竹北市,竹北市縣華段142號,立體,小型車758格,計時、月租,NaN,758,0,24.834687,120.993368
1,1,003-3,湖口鄉公所,03-5993911-292,湖口鄉中興立體停車場,湖口鄉,湖口鄉興湖段58-74等17筆號,立體,小型車147格,計時、月租,NaN,147,0,24.899005,121.039578
2,2,007-8,國雲公寓大廈管理維護股份有限公司,02-25573722,竹東仁愛路公有停車場(廣停二),竹東鎮,新竹縣竹東鎮仁愛路/學前路口(竹東鎮仁愛段1252號,平面,小型車163格,計時、月租,NaN,163,0,24.734838,121.090595
3,3,008-7,國雲公寓大廈管理維護股份有限公司,02-25573722,華興街立體停車場,竹北市,竹北市華興段129號,立體,小型車135格,計時、月租,NaN,135,0,24.839478,121.001994
4,4,010-7,國雲公寓大廈管理維護股份有限公司,02-25573722,中正東路停車場,竹北市,竹北市竹仁段1024號,平面,小型車45格,計時、月租,NaN,45,0,24.837015,121.012190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,92,?亭停車事業股份有限公司,02-27888750-8813,道禾二平面停車場,竹北市,竹北市自強段167號,平面,小型車41格,計時、月租,NaN,41,0,24.834687,120.993368
64,64,93,巨人社區管理委員會,03-6580380,巨人社區停車場,竹北市,竹北市莊敬段269號,平面,小型車62格,計時、月租,NaN,62,0,24.818510,121.020833
65,65,94,大榆二生活建築有限公司,936321599,打里摺．大榆二社區附設停車場,竹北市,竹北市永興段481號,平面,小型車35格,計時,NaN,35,0,24.834687,120.993368
66,66,95,陳信源,02-27888750-8813黃小姐,竹北新國停車場,竹北市,竹北市新國段824號,平面,小型車19格,計時、月租,NaN,19,0,24.840851,120.999048


In [169]:
park_df = park_df.rename(columns={'停車場名稱':'NAME', '停車場地址-地號':'ADDRESS'})
park_df = park_df[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [173]:
park_df.loc[48]

NAME         新竹縣游泳館停車場
ADDRESS       公園段1164號
CAR_NUM             63
MOTOR_NUM            0
lat                NaN
lng                NaN
Name: 48, dtype: object

In [174]:
park_df.loc[48, 'lat'] = 24.8209348
park_df.loc[48, 'lng'] = 121.0205708

In [176]:
park_df.to_csv('停車/新竹停車場/新竹縣停車場(processed).csv', index=False, encoding='utf-8-sig')

In [178]:
park_df1 = pd.read_csv('停車/新竹停車場/新竹縣停車場(processed).csv', encoding='utf-8')
park_df2 = pd.read_csv('停車/新竹停車場/新竹市停車場(processed).csv', encoding='utf-8')

In [181]:
park_df = pd.concat([park_df1, park_df2], axis=0).reset_index(drop=True)

In [182]:
park_df.to_csv('停車/新竹停車場/新竹停車場(processed).csv', index=False, encoding='utf-8-sig')

## 桃園

In [183]:
park_df = pd.read_csv('停車/桃園停車場/桃園停車場.csv', encoding='utf-8')

In [185]:
park_df['地址'] = park_df['地址'].str.split('、').str[0] + '號'
park_df['地址'] = park_df['地址'].str.split('地號').str[0] + '號'
park_df['地址'] = park_df['地址'].str.split('號').str[0] + '號'

In [186]:
park_df

,停車場名稱,形式,小型車,大型車,機車,營業時間,收費標準,地址,電話
0,0.0,0.0,0,0,0,0.0,0.0,0.0號,0.0
1,西門地下停車場,立體停車場,548,0,0,00:00~24:00,小型車-臨停:30/時小型車-月租:3600/月,桃園區四維路39號,03-3310804、0800-021-111
2,桃園市公有府前地下停車場,立體停車場,334,0,76,00:00~24:00(),小型車-臨停:30/時小型車-月租:3600/月機車-月租:600/月,桃園區縣府路1號,03-3473050
3,新民立體停車場,立體停車場,191,0,0,00:00~24:00(),小型車-臨停:30/時小型車-月租:3600/月,桃園市桃園區民權路32號,02-24551299
4,衛生福利部桃園醫院停車場,平面停車場,151,0,332,00:00~24:00(),小型車-臨停:20/時小型車-臨停:30/時機車-臨停:10/次機車-臨停:20/次,桃園市桃園區中山路1492號,02-22461708
...,...,...,...,...,...,...,...,...,...
454,正好停福州站,平面停車場,11,0,0,00:00~24:00(),小型車-臨停:60/時小型車-月租:5000/月,中壢區福州路243號,02-27995657
455,文東五停車場,平面停車場,32,0,0,00:00~24:00(),小型車-臨停:20/時小型車-月租:3000/月,桃園市龜山區文東五街117巷30號,NaN
456,桃園市大園區農會本會停車場,平面停車場,39,0,0,00:00~24:00(),小型車-臨停:40/時,桃園市大園區中正東路103號,NaN
457,桃園市大園區農會三和停車場,平面停車場,50,0,0,00:00~24:00(),小型車-臨停:40/時,桃園市大園區三石里福德路8鄰1號,NaN


In [189]:
for i, address in enumerate(park_df['地址'].values): 
    index = i
    if (i+1)%50 == 0:
        print('Already process %d geocoding requests.'%(i+1))
    geocode_result = gmaps.geocode(address, region='tw')
    try:
        park_df.loc[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
        park_df.loc[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']
    except:
        print('*error at index %d.'%(i))

*error at index 0.
*error at index 21.
Already process 50 geocoding requests.
Already process 100 geocoding requests.
*error at index 103.
Already process 150 geocoding requests.
Already process 200 geocoding requests.
*error at index 228.
Already process 250 geocoding requests.
Already process 300 geocoding requests.
*error at index 341.
Already process 350 geocoding requests.
*error at index 356.
*error at index 371.
Already process 400 geocoding requests.
Already process 450 geocoding requests.


In [198]:
park_df.loc[371]

停車場名稱                     昇捷建設停車場
形式                          平面停車場
小型車                           100
大型車                             0
機車                              0
營業時間                00:00~24:00()
收費標準     小型車-臨停:30/時小型車-月租:2500/時
地址                         同德段41號
電話                 03-3028989#712
lat                           NaN
lng                           NaN
Name: 371, dtype: object

In [199]:
park_df.loc[21, 'lat'] = 24.9889297
park_df.loc[21, 'lng'] = 121.2602597
park_df.loc[103, 'lat'] = 24.980334
park_df.loc[103, 'lng'] = 121.271316
park_df.loc[228, 'lat'] = 25.021431
park_df.loc[228, 'lng'] = 121.314477
park_df.loc[341, 'lat'] = 25.031509
park_df.loc[341, 'lng'] = 121.295511
park_df.loc[356, 'lat'] = 24.991927
park_df.loc[356, 'lng'] = 121.280035
park_df.loc[371, 'lat'] = 25.0222542
park_df.loc[371, 'lng'] = 121.2947988

In [204]:
park_df = park_df.drop([0],axis=0).reset_index(drop=True)

In [205]:
park_df

,停車場名稱,形式,小型車,大型車,機車,營業時間,收費標準,地址,電話,lat,lng
0,西門地下停車場,立體停車場,548,0,0,00:00~24:00,小型車-臨停:30/時小型車-月租:3600/月,桃園區四維路39號,03-3310804、0800-021-111,24.959669,121.230776
1,桃園市公有府前地下停車場,立體停車場,334,0,76,00:00~24:00(),小型車-臨停:30/時小型車-月租:3600/月機車-月租:600/月,桃園區縣府路1號,03-3473050,24.992903,121.301060
2,新民立體停車場,立體停車場,191,0,0,00:00~24:00(),小型車-臨停:30/時小型車-月租:3600/月,桃園市桃園區民權路32號,02-24551299,24.992895,121.310800
3,衛生福利部桃園醫院停車場,平面停車場,151,0,332,00:00~24:00(),小型車-臨停:20/時小型車-臨停:30/時機車-臨停:10/次機車-臨停:20/次,桃園市桃園區中山路1492號,02-22461708,24.977703,121.269191
4,衛生福利部桃園療養院停車場,平面停車場,219,0,0,00:00~24:00(),小型車-臨停:20/時小型車-臨停:30/時,桃園市桃園區龍壽街71號,02-22461708,24.979546,121.268890
...,...,...,...,...,...,...,...,...,...,...,...
453,正好停福州站,平面停車場,11,0,0,00:00~24:00(),小型車-臨停:60/時小型車-月租:5000/月,中壢區福州路243號,02-27995657,24.948969,121.234860
454,文東五停車場,平面停車場,32,0,0,00:00~24:00(),小型車-臨停:20/時小型車-月租:3000/月,桃園市龜山區文東五街117巷30號,NaN,25.050999,121.362488
455,桃園市大園區農會本會停車場,平面停車場,39,0,0,00:00~24:00(),小型車-臨停:40/時,桃園市大園區中正東路103號,NaN,25.062865,121.201668
456,桃園市大園區農會三和停車場,平面停車場,50,0,0,00:00~24:00(),小型車-臨停:40/時,桃園市大園區三石里福德路8鄰1號,NaN,25.074384,121.258996


In [206]:
park_df = park_df.rename(columns={'停車場名稱':'NAME', '地址':'ADDRESS', '小型車':'CAR_NUM', '機車':'MOTOR_NUM'})
park_df = park_df[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [207]:
park_df

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,西門地下停車場,桃園區四維路39號,548,0,24.959669,121.230776
1,桃園市公有府前地下停車場,桃園區縣府路1號,334,76,24.992903,121.301060
2,新民立體停車場,桃園市桃園區民權路32號,191,0,24.992895,121.310800
3,衛生福利部桃園醫院停車場,桃園市桃園區中山路1492號,151,332,24.977703,121.269191
4,衛生福利部桃園療養院停車場,桃園市桃園區龍壽街71號,219,0,24.979546,121.268890
...,...,...,...,...,...,...
453,正好停福州站,中壢區福州路243號,11,0,24.948969,121.234860
454,文東五停車場,桃園市龜山區文東五街117巷30號,32,0,25.050999,121.362488
455,桃園市大園區農會本會停車場,桃園市大園區中正東路103號,39,0,25.062865,121.201668
456,桃園市大園區農會三和停車場,桃園市大園區三石里福德路8鄰1號,50,0,25.074384,121.258996


In [208]:
park_df.to_csv('停車/桃園停車場/桃園停車場(processed).csv', index=False, encoding='utf-8-sig')

## 台中
* 滿多座標對應是手動弄的 因為資料太亂很難只用程式處理

In [233]:
park_df1 = pd.read_csv('停車/台中停車場/臺中市公有路外停車場總表.csv.CSV', encoding='utf-8')
park_df2 = pd.read_csv('停車/台中停車場/臺中市停車場座標位置.CSV', encoding='utf-8')
park_df3 = pd.read_csv('停車/台中停車場/臺中市民營停車場資料.CSV', encoding='utf-8')

In [213]:
park_df1.head(2)

,編號,區別,名稱,車位數_大客車,車位數_汽車,車位數_機車,土地面積_㎡,設置地點,構造方式,收費費率,收費時間,代管單位,備註
0,1,中,(停34)自由路 (第一期)立體停車場,NaN,368.0,NaN,4687.0,自由路、公園路(台中市自由路2段92號),"地上七層RC匝道式,地下一層",30元/時\n以30分鐘計費,每日0:00~24:00,委託民間經營管理,NaN
1,2,中,大誠立體停車場 (柳川舊河道),NaN,272.0,NaN,2933.0,成功路、大誠街口(台中市吉祥街20號),地上三層鋼架匝道式,10元/30分鐘,每日0:00~24:00,自營,NaN


In [214]:
park_df2.head(2)

,停車場,緯度,經度,位置
0,(公2-2)市政公園地下停車場,24.15938,120.64716,臺中市西屯區文心路與惠中路間
1,(公43)府後街地下停車場,24.13624,120.67508,臺中市西區府後街1之8號


In [235]:
for idx, row in park_df1.iterrows():
    #print(row.名稱)
    condition = park_df2['停車場'] == row.名稱
    try:
        park_df1.loc[idx, 'lat'] = park_df2.loc[condition, '緯度'].values
        park_df1.loc[idx, 'lng'] = park_df2.loc[condition, '經度'].values
        print('success idx: ',idx)
    except:
        pass
        #print('error in idx:%d.'%(idx))

success idx:  1
success idx:  2
success idx:  5
success idx:  6
success idx:  7
success idx:  10
success idx:  14
success idx:  15
success idx:  16
success idx:  19
success idx:  21
success idx:  25
success idx:  28
success idx:  30
success idx:  31
success idx:  32
success idx:  36
success idx:  41
success idx:  44
success idx:  45
success idx:  46
success idx:  48
success idx:  49
success idx:  50
success idx:  52
success idx:  53
success idx:  55
success idx:  56
success idx:  64
success idx:  66
success idx:  75
success idx:  77
success idx:  78
success idx:  79
success idx:  80
success idx:  81
success idx:  82
success idx:  83
success idx:  84
success idx:  87
success idx:  88
success idx:  89
success idx:  92
success idx:  93
success idx:  94
success idx:  95
success idx:  97
success idx:  98
success idx:  99
success idx:  101
success idx:  102
success idx:  103
success idx:  106
success idx:  107
success idx:  108
success idx:  111
success idx:  112
success idx:  113
success id

In [236]:
park_df1

,編號,區別,名稱,車位數_大客車,車位數_汽車,車位數_機車,土地面積_㎡,設置地點,構造方式,收費費率,收費時間,代管單位,備註,lat,lng
0,1,中,(停34)自由路 (第一期)立體停車場,NaN,368.0,NaN,4687.0,自由路、公園路(台中市自由路2段92號),"地上七層RC匝道式,地下一層",30元/時\n以30分鐘計費,每日0:00~24:00,委託民間經營管理,NaN,NaN,NaN
1,2,中,大誠立體停車場 (柳川舊河道),NaN,272.0,NaN,2933.0,成功路、大誠街口(台中市吉祥街20號),地上三層鋼架匝道式,10元/30分鐘,每日0:00~24:00,自營,NaN,24.14469,120.67930
2,3,中,(停103)興中停車場,NaN,141.0,NaN,10095.0,大誠街、中正路口,平面式,10元/30分鐘,每日08:00~18:00,自營,NaN,24.14418,120.67886
3,4,西,(機51)州廳前廣場停車場,NaN,58.0,NaN,1476.0,民權路、市府路,平面式,累進費率 （30分鐘離場免費）(以30分鐘計費),機關上班日08:00~18:00,自營,NaN,NaN,NaN
4,5,西,(停73)大益停車場,NaN,78.0,NaN,1189.0,向上路 、大忠街,立體式,未收費,NaN,未收費,重建中,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,297,大肚,停五停車場,NaN,73.0,NaN,2281.0,大德五街旁,平面式,NaN,NaN,本府未收費管理,NaN,NaN,NaN
297,298,大肚,廣停六停車場,NaN,16.0,NaN,4500.0,頂街里沙田路上，運動公園旁,平面式,NaN,NaN,本府未收費管理,NaN,NaN,NaN
298,299,大肚,(停2)停車場,NaN,24.0,0.0,1000.0,大肚區沙田路一段428巷與大肚區沙田路一段428巷9弄口,平面式,0,0,公所未收費管理,變更高速公路王田交流道附近特定區計畫,NaN,NaN
299,300,大肚,大肚地下停車場,NaN,123.0,54.0,NaN,大肚區自治路81號,立體式,20元/時\n以30分鐘計費\n月租1200元,每日\n0:00~24:00,公所委託民間經營管理,NaN,24.15135,120.54634


In [238]:
park_df1.to_csv('停車/台中停車場/臺中市公有路外停車場總表_.csv.CSV', index=False, encoding='utf-8-sig')

In [243]:
park_df1 = pd.read_csv('停車/台中停車場/臺中市公有路外停車場總表_.csv.CSV', encoding='ansi', sep='\t')

In [245]:
park_df1 = park_df1.drop(['區別.1', '名稱.1'], axis=1)

In [250]:
park_df1 = park_df1.rename(columns={'名稱':'NAME', '設置地點':'ADDRESS', ' 車位數_汽車':'CAR_NUM', ' 車位數_機車':'MOTOR_NUM'})
park_df1 = park_df1[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [252]:
park_df1['CAR_NUM'] = park_df1['CAR_NUM'].fillna(0)
park_df1['MOTOR_NUM'] = park_df1['MOTOR_NUM'].fillna(0)
park_df1

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,(停34)自由路 (第一期)立體停車場,自由路、公園路(台中市自由路2段92號),368.0,0.0,24.142637,120.684819
1,大誠立體停車場 (柳川舊河道),成功路、大誠街口(台中市吉祥街20號),272.0,0.0,24.144690,120.679300
2,(停103)興中停車場,大誠街、中正路口,141.0,0.0,24.144180,120.678860
3,(機51)州廳前廣場停車場,民權路、市府路,58.0,0.0,24.138949,120.678725
4,(停73)大益停車場,向上路 、大忠街,78.0,0.0,24.146225,120.654294
...,...,...,...,...,...,...
296,停五停車場,大德五街旁,73.0,0.0,24.153920,120.540630
297,廣停六停車場,頂街里沙田路上，運動公園旁,16.0,0.0,24.157630,120.543760
298,(停2)停車場,大肚區沙田路一段428巷與大肚區沙田路一段428巷9弄口,24.0,0.0,24.124312,120.565783
299,大肚地下停車場,大肚區自治路81號,123.0,54.0,24.151350,120.546340


In [253]:
park_df1.to_csv('停車/台中停車場/臺中市公有路外停車場總表(processed).csv.CSV', index=False, encoding='utf-8-sig')

In [255]:
park_df3['停車場位置'] = park_df3['停車場位置'].str.split('、').str[0] + '號'
park_df3['停車場位置'] = park_df3['停車場位置'].str.split('地號').str[0] + '號'
park_df3['停車場位置'] = park_df3['停車場位置'].str.split('號').str[0] + '號'

In [256]:
park_df3

,年度,編號,區別,核准字號,停車場名稱,負責人,公司名稱,電話,通訊錄,建物內容停車場形式,...,停車場位置,土地標示,大型車,小型車,身障汽車格位,機車,身障機車格位,使用期限,公營或私營,備註
0,97.0,1.0,北區,(97)府交停證字第023號,中國醫藥大學五權立體停車場,林正介,中國醫藥大學附設醫院,(04)22052121,臺中市北區五常里13鄰五權路499號,立體式,...,臺中市北區五常里13鄰五權路499號,北區水源段245-0地號,0.0,511.0,11.0,635.0,13.0,97/09/04-117/09/03,私,收費標準：汽車臨時停車每小時新台幣40元，停車時數未滿一小時者，以一小時計算收費，停車時數逾...
1,101.0,2.0,西屯,(101)中市交停規證字第062號,台中榮民總醫院門診立體停車場,李三剛,行政院國軍退除役官兵輔導委員會台中榮民總醫院,(04)23592525\n#2484,407臺中市西屯區台中港路3段160號,立體式,...,407臺中市西屯區台中港路3段160號,西屯區國安段261、262、347、370地號等4筆土地,0.0,324.0,8.0,0.0,0.0,101/11/6-108/04/30,私,(98)府交停證字第003號\n小型車30元/hr
2,101.0,3.0,西屯,(101)中市交停規證字第063號,台中榮民總醫院4號門立體停車場,李三剛,行政院國軍退除役官兵輔導委員會台中榮民總醫院,(04)23592525\n#2484,407臺中市西屯區台中港路3段160號,立體式,...,407臺中市西屯區台中港路3段160號,西屯區國安段261、262、347、370地號等4筆土地,0.0,380.0,8.0,0.0,0.0,101/11/6-108/04/30,私,(98)府交停證字第004-1號\n小型車30元/hr
3,102.0,4.0,西區,(102)中市交停規證字第032號,金典綠園道停車場,王俊賢,日華金典國際酒店股份有限公司,04-23288000,403台中市西區健行路1049號,停車塔,...,台中市西區博館3街30號,台中市西區麻園頭段15-80、15-81、15-82地號等3筆土地,0.0,100.0,0.0,0.0,0.0,自102年7月10日起至107年5月31日止。(102/07/31變更負責人),私,計時：平日20元/半小時；假日30元/半小時(假日為周六、日及國定假日)。(102/07/3...
4,102.0,5.0,北屯區,(102)中市交停規證字第051號,幸福崇德停車場,李慶輝,NaN,0932-625947,408台中市南屯區大墩路20巷15號,平面式,...,台中市北屯區河北一街與河北東街口號,台中市北屯區崇德段416地號等1筆土地,0.0,41.0,2.0,0.0,0.0,自102年10月29日起至107年10月30日止,私,計時：汽車臨時停車每半小時20元，停車時數未滿半小時者，以半小時計算收費。汽車月租每月每輛新...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,107.0,177.0,西屯區,(107)中市交停規證字第028號,吳駿麒停車場,吳駿麒 君,NaN,0937-552269,433台中市沙鹿區鹿寮里成功東街69號,平面式,...,台中市西屯區福順路65號,台中市西屯區福順段165地號共1筆土地,0.0,25.0,1.0,0.0,0.0,自107年4月12日起至111年10月31日止。,私,"收費標準：(1)汽車臨時停車每30分鐘新台幣20元。汽車月租每月新台幣2,500元。(2)凡..."
177,107.0,178.0,西屯區,(107)中市交停規證字第029號,Times優衣庫台中福科路店停車場,牧野 純 君,普客二四停車場股份有限公司,02-25046098 0800-888-924,10486台北市中山區松江路87號3F,平面式,...,台中市西屯區福科路186號,台中市西屯區福順段92、93、95、96、97、99、100地號共7筆土地,0.0,35.0,1.0,0.0,0.0,自107年4月12日起至112年4月11日止,私,收費標準：汽車臨時停車每小時新台幣40元，停車時數未滿一小時者，以一小時計算收費，停車時數逾...
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
for i, address in enumerate(park_df3['停車場位置'].values): 
    index = i
    if (i+1)%50 == 0:
        print('Already process %d geocoding requests.'%(i+1))
    geocode_result = gmaps.geocode(address, region='tw')
    try:
        park_df3.loc[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
        park_df3.loc[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']
    except:
        print('*error at index %d.'%(i))

Already process 50 geocoding requests.
Already process 100 geocoding requests.
*error at index 101.
*error at index 109.
*error at index 110.
*error at index 140.
Already process 150 geocoding requests.


In [267]:
park_df3.loc[140]

年度                                                           106
編號                                                           141
區別                                                           西屯區
核准字號                                           (106)中市交停規證字第091號
停車場名稱                                                      華廈停車場
負責人                                                       馮孟珠  君
公司名稱                                                   捷報停車場有限公司
電話                                                   04-23161649
通訊錄                                     407台中市西屯區華美西街二段311號13F-2
建物內容停車場形式                                                    平面式
使用層樓面積_平方公尺                                               466.87
停車場位置                                       台中市西屯區福星路與西安街143巷旁空號
土地標示                                         臺中市西屯區福星段207地號等1筆土地
大型車                                                            0
小型車                                                           11
身障汽車格位                   

In [268]:
park_df3.loc[101, 'lat'] = 24.1462211
park_df3.loc[101, 'lng'] = 120.6214632

park_df3.loc[109, 'lat'] = 24.1822277
park_df3.loc[109, 'lng'] = 120.643831

park_df3.loc[110, 'lat'] = 24.1822277
park_df3.loc[110, 'lng'] = 120.643831

park_df3.loc[140, 'lat'] = 24.1822277
park_df3.loc[140, 'lng'] = 120.643831

In [269]:
park_df3

,年度,編號,區別,核准字號,停車場名稱,負責人,公司名稱,電話,通訊錄,建物內容停車場形式,...,大型車,小型車,身障汽車格位,機車,身障機車格位,使用期限,公營或私營,備註,lat,lng
0,97.0,1.0,北區,(97)府交停證字第023號,中國醫藥大學五權立體停車場,林正介,中國醫藥大學附設醫院,(04)22052121,臺中市北區五常里13鄰五權路499號,立體式,...,0.0,511.0,11.0,635.0,13.0,97/09/04-117/09/03,私,收費標準：汽車臨時停車每小時新台幣40元，停車時數未滿一小時者，以一小時計算收費，停車時數逾...,24.154402,120.684439
1,101.0,2.0,西屯,(101)中市交停規證字第062號,台中榮民總醫院門診立體停車場,李三剛,行政院國軍退除役官兵輔導委員會台中榮民總醫院,(04)23592525\n#2484,407臺中市西屯區台中港路3段160號,立體式,...,0.0,324.0,8.0,0.0,0.0,101/11/6-108/04/30,私,(98)府交停證字第003號\n小型車30元/hr,24.188526,120.619118
2,101.0,3.0,西屯,(101)中市交停規證字第063號,台中榮民總醫院4號門立體停車場,李三剛,行政院國軍退除役官兵輔導委員會台中榮民總醫院,(04)23592525\n#2484,407臺中市西屯區台中港路3段160號,立體式,...,0.0,380.0,8.0,0.0,0.0,101/11/6-108/04/30,私,(98)府交停證字第004-1號\n小型車30元/hr,24.188526,120.619118
3,102.0,4.0,西區,(102)中市交停規證字第032號,金典綠園道停車場,王俊賢,日華金典國際酒店股份有限公司,04-23288000,403台中市西區健行路1049號,停車塔,...,0.0,100.0,0.0,0.0,0.0,自102年7月10日起至107年5月31日止。(102/07/31變更負責人),私,計時：平日20元/半小時；假日30元/半小時(假日為周六、日及國定假日)。(102/07/3...,24.156177,120.664362
4,102.0,5.0,北屯區,(102)中市交停規證字第051號,幸福崇德停車場,李慶輝,NaN,0932-625947,408台中市南屯區大墩路20巷15號,平面式,...,0.0,41.0,2.0,0.0,0.0,自102年10月29日起至107年10月30日止,私,計時：汽車臨時停車每半小時20元，停車時數未滿半小時者，以半小時計算收費。汽車月租每月每輛新...,24.176525,120.685020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,107.0,177.0,西屯區,(107)中市交停規證字第028號,吳駿麒停車場,吳駿麒 君,NaN,0937-552269,433台中市沙鹿區鹿寮里成功東街69號,平面式,...,0.0,25.0,1.0,0.0,0.0,自107年4月12日起至111年10月31日止。,私,"收費標準：(1)汽車臨時停車每30分鐘新台幣20元。汽車月租每月新台幣2,500元。(2)凡...",24.180481,120.625780
177,107.0,178.0,西屯區,(107)中市交停規證字第029號,Times優衣庫台中福科路店停車場,牧野 純 君,普客二四停車場股份有限公司,02-25046098 0800-888-924,10486台北市中山區松江路87號3F,平面式,...,0.0,35.0,1.0,0.0,0.0,自107年4月12日起至112年4月11日止,私,收費標準：汽車臨時停車每小時新台幣40元，停車時數未滿一小時者，以一小時計算收費，停車時數逾...,24.181806,120.626013
178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.737222,120.973889
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.737222,120.973889


In [271]:
park_df3 = park_df3.drop([178,179,180], axis=0)

In [273]:
park_df3.columns

Index(['年度', '編號', '區別', '核准字號', '停車場名稱', '負責人', '公司名稱', '電話', '通訊錄',
       '建物內容停車場形式', '使用層樓面積_平方公尺', '停車場位置', '土地標示', '大型車', '小型車', '身障汽車格位',
       '機車', '身障機車格位', '使用期限', '公營或私營', '備註', 'lat', 'lng'],
      dtype='object')

In [274]:
park_df3 = park_df3.rename(columns={'停車場名稱':'NAME', '停車場位置':'ADDRESS', '小型車':'CAR_NUM', '機車':'MOTOR_NUM'})
park_df3 = park_df3[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [275]:
park_df3

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,中國醫藥大學五權立體停車場,臺中市北區五常里13鄰五權路499號,511.0,635.0,24.154402,120.684439
1,台中榮民總醫院門診立體停車場,407臺中市西屯區台中港路3段160號,324.0,0.0,24.188526,120.619118
2,台中榮民總醫院4號門立體停車場,407臺中市西屯區台中港路3段160號,380.0,0.0,24.188526,120.619118
3,金典綠園道停車場,台中市西區博館3街30號,100.0,0.0,24.156177,120.664362
4,幸福崇德停車場,台中市北屯區河北一街與河北東街口號,41.0,0.0,24.176525,120.685020
...,...,...,...,...,...,...
173,Times台中文心一路停車場,台中市西屯區文心一路156號,31.0,0.0,24.154783,120.642115
174,聯永物業股份有限公司台中十五站停車場,臺中市西區公正路181號,7.0,0.0,24.148881,120.660511
175,台灣聯通台中逢甲停車場,台中市西屯區逢甲路221號,18.0,0.0,24.176942,120.641491
176,吳駿麒停車場,台中市西屯區福順路65號,25.0,0.0,24.180481,120.625780


In [280]:
park_df3.to_csv('停車/台中停車場/臺中市民營停車場資料(processed).csv', index=False, encoding='utf-8-sig')

In [284]:
park_df1 = pd.read_csv('停車/台中停車場/臺中市公有路外停車場總表(processed).csv.CSV', encoding='utf-8')
park_df2 = pd.read_csv('停車/台中停車場/臺中市民營停車場資料(processed).CSV', encoding='utf-8')
park_df = pd.concat([park_df1, park_df2], axis=0).reset_index(drop=True)

In [285]:
park_df

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,(停34)自由路 (第一期)立體停車場,自由路、公園路(台中市自由路2段92號),368.0,0.0,24.142637,120.684819
1,大誠立體停車場 (柳川舊河道),成功路、大誠街口(台中市吉祥街20號),272.0,0.0,24.144690,120.679300
2,(停103)興中停車場,大誠街、中正路口,141.0,0.0,24.144180,120.678860
3,(機51)州廳前廣場停車場,民權路、市府路,58.0,0.0,24.138949,120.678725
4,(停73)大益停車場,向上路 、大忠街,78.0,0.0,24.146225,120.654294
...,...,...,...,...,...,...
474,Times台中文心一路停車場,台中市西屯區文心一路156號,31.0,0.0,24.154783,120.642115
475,聯永物業股份有限公司台中十五站停車場,臺中市西區公正路181號,7.0,0.0,24.148881,120.660511
476,台灣聯通台中逢甲停車場,台中市西屯區逢甲路221號,18.0,0.0,24.176942,120.641491
477,吳駿麒停車場,台中市西屯區福順路65號,25.0,0.0,24.180481,120.625780


In [286]:
park_df.to_csv('停車/台中停車場/台中市停車場(processed).csv', index=False, encoding='utf-8-sig')

## 台南

In [317]:
park_df1 = pd.read_csv('停車/臺南停車場/臺南停車場1.csv', encoding='utf-8')
park_df2 = pd.read_csv('停車/臺南停車場/臺南停車場2.csv', encoding='utf-8')
park_df3 = pd.read_csv('停車/臺南停車場/臺南停車場3.csv', encoding='utf-8')

In [318]:
park_df1

,停車場型態,停車場代碼,停車場名稱,停車場地址,即時車位,一般大型車,一般小型車,身障者小型車,婦幼者小型車,綠能小型車,一般機車,身障者機車,收費時間,收費費率,經緯度
0,民營停車場,101091,赤崁樓停車場,臺南市中西區民族路2段212號,-1,0,92,1,0,0,0,0,上午8時至晚上10時,臨停費率:小型車每小時30元，超過1小時，超出時間未達30分鐘加收15元；大型車每小時100...,22.996785，120.202848
1,民營停車場,103013,崇善停車場,臺南市東區崇善路913巷6弄口（德高段349地號）,-1,0,4,1,0,0,0,0,24小時,月租部份:每月每車1，800元,22.965755，120.233832
2,民營停車場,103025,和益停車場,臺南市永康區永康車站旁,-1,0,0,0,0,0,76,2,早上5時30分至淩晨0時30分,機車:每日每次20元，月租每月400元。,23.038336，120.253523
3,民營停車場,103031,台灣聯通民生場,臺南市中西區民生路2段279號,-1,0,132,3,2,1,0,0,24小時,臨停費率:每小時60元，停車未滿1小時以1小時計費，停車逾1小時以上者，未滿半小時以半小時計...,22.996694，120.192082
4,民營停車場,103035,凱得停車場,臺南市永康區國光八街108號旁空地（大灣段6829、6832地號）,-1,0,20,1,0,0,0,0,24小時,月租年繳15，000元，半年繳8，000元。,23.011418，120.259606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,民營停車場,108095,竑穗永福停車場,臺南市南區臺南市南區健康路1段近永福路口（鹽埕段1-656、1-657、169-2、168-...,-1,0,31,1,1,1,0,0,24小時,平日第1小時60元，之後每半小時加收30元，當日最高收費600元; 假日第1小時70元，之後...,22.981218，120.200060
230,民營停車場,108096,Times燦坤台南永華二店停車場,臺南市安平區臺南是安平區永華路二段35號,-1,0,9,1,1,1,0,0,24小時,半小時30元,22.989024，120.185767
231,民營停車場,A01351,博客停車場開發股份有限公司台南轉運站停車場,臺南市北區臺南市北區公園路291號（北華段21、131地號）,182,0,229,8,8,16,449,17,24小時,小型車臨停每小時20元，月租3，000元；機車臨停每小時10元，月租1，500元。(臨停皆以...,23.001679，120.209027
232,民營停車場,B00009,博客停車場開發股份有限公司燦坤海佃場,臺南市安南區臺南市安南區海佃路1段329號,68,0,146,4,4,1,0,0,24小時,平日：每小時30元 假日：每小時50元 月租：3000元/月,23.026049，120.191006


In [292]:
park_df2.head(2)

,停車場型態,停車場代碼,停車場名稱,停車場地址,即時車位,一般大型車,一般小型車,身障者小型車,婦幼者小型車,綠能小型車,一般機車,身障者機車,收費時間,收費費率,經緯度
0,公有免費停車場,04,大林路111巷公有停車場(停11),臺南市南區大林路111巷、大同路2段482巷72弄中間,-1,0,28,1,0,0,0,0,NaN,NaN,22.974261，120.21202
1,公有免費停車場,06,安平港鯤鯓站臨時免費公有停車場,臺南市南區安平港聯外道路(四鯤鯓碼頭管制站前),-1,0,85,2,0,2,13,0,NaN,NaN,22.966607，120.16586


In [293]:
park_df3.head(2)

,停車場型態,停車場代碼,停車場名稱,停車場地址,即時車位,一般大型車,一般小型車,身障者小型車,婦幼者小型車,綠能小型車,一般機車,身障者機車,收費時間,收費費率,經緯度
0,公有收費停車場,03,延平郡王祠大埔街口路外停車場,臺南市中西區開山路與大埔街路口西側,-1,0,6,1,0,1,0,0,08:00~22:00,20元/時,22.987274，120.208698
1,公有收費停車場,16,臺南市立棒球場北側路外停車場,臺南市南區健康路1段市立棒球場北側,-1,0,21,1,0,2,0,0,08:00~22:00,20元/時,22.981203，120.205877


In [301]:
park_df1.經緯度.str.split('，').values

array([list(['22.996785', '120.202848']),
       list(['22.965755', '120.233832']),
       list(['23.038336', '120.253523']),
       list(['22.996694', '120.192082']),
       list(['23.011418', '120.259606']),
       list(['22.992103', '120.233264']),
       list(['22.989882', '120.233606']),
       list(['22.997683', '120.206074']),
       list(['23.336388', '120.504550']),
       list(['23.336390', '120.504326']),
       list(['22.998513', '120.227428']),
       list(['22.998593', '120.226838']),
       list(['23.180225', '120.232857']),
       list(['22.968726', '120.226791']),
       list(['22.975339', '120.220763']),
       list(['22.995506', '120.206413']),
       list(['23.006501', '120.233034']),
       list(['22.999063', '120.208996']),
       list(['23.301409', '120.310874']),
       list(['22.925709', '120.284688']),
       list(['23.012056', '120.208761']),
       list(['22.999756', '120.227488']),
       list(['23.012586', '120.194221']),
       list(['22.983470', '120.234

In [310]:
park_df1.loc[0].經緯度.split('，')[0]

'22.996785'

In [322]:
for idx, row in park_df1.iterrows():
    l = row.經緯度.split('，')
    park_df1.loc[idx, 'lat'] = l[0]
    park_df1.loc[idx, 'lng'] = l[1]

In [325]:
park_df3

,停車場型態,停車場代碼,停車場名稱,停車場地址,即時車位,一般大型車,一般小型車,身障者小型車,婦幼者小型車,綠能小型車,一般機車,身障者機車,收費時間,收費費率,經緯度,lat,lng
0,公有收費停車場,03,延平郡王祠大埔街口路外停車場,臺南市中西區開山路與大埔街路口西側,-1,0,6,1,0,1,0,0,08:00~22:00,20元/時,22.987274，120.208698,22.987274,120.208698
1,公有收費停車場,16,臺南市立棒球場北側路外停車場,臺南市南區健康路1段市立棒球場北側,-1,0,21,1,0,2,0,0,08:00~22:00,20元/時,22.981203，120.205877,22.981203,120.205877
2,公有收費停車場,19,市立圖書館廣場停車場,臺南市北區公園路市立圖書館前,-1,0,56,1,0,2,0,0,08:00~22:00,20元/時,23.004306，120.211019,23.004306,120.211019
3,公有收費停車場,23,勝安香草公園路外停車場,臺南市北區開元路63巷勝安香草公園北側,-1,0,16,1,0,1,0,0,08:00~22:00,20元/時,23.006637，120.215783,23.006637,120.215783
4,公有收費停車場,31,新營機車停車場,臺南市新營區新營火車站西南側,-1,0,0,0,0,0,186,3,24小時,40元/次,23.306363，120.322730,23.306363,120.322730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,公有收費停車場,B00012,永康中華立體停車場,臺南市永康區華興街77號,67,0,274,5,5,5,0,0,24小時,20元/時,23.000707，120.237698,23.000707,120.237698
80,公有收費停車場,B00013,永康區復國里停(七)停車場,臺南市永康區復國一路150巷26號旁,25,0,43,1,1,0,0,0,24小時,20元/時,23.005257，120.247223,23.005257,120.247223
81,公有收費停車場,B00014,仁德立體停車場(傳忠有限公司),臺南市仁德區臺南市仁德區正義一街41號,44,0,201,6,4,1,0,0,24小時,臨停:第一小時20元，第二小時起未滿半小時者，以半小時計算；逾半小時以上，如不逾15分鐘者以...,22.972604，120.253466,22.972604,120.253466
82,公有收費停車場,B00015,新市國小地下停車場,臺南市新市區中興路1號B1,24,0,154,3,3,1,0,0,24小時,臨停:每小時20元，前30分鐘免費，第1小時以1小時計，逾1小時以上以半小時計算。 月租:全...,23.078299，120.2953306,23.078299,120.2953306


In [326]:
park_df1 = park_df1.rename(columns={'停車場名稱':'NAME', '停車場地址':'ADDRESS', '一般小型車':'CAR_NUM', '一般機車':'MOTOR_NUM'})
park_df1 = park_df1[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

park_df2 = park_df2.rename(columns={'停車場名稱':'NAME', '停車場地址':'ADDRESS', '一般小型車':'CAR_NUM', '一般機車':'MOTOR_NUM'})
park_df2 = park_df2[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

park_df3 = park_df3.rename(columns={'停車場名稱':'NAME', '停車場地址':'ADDRESS', '一般小型車':'CAR_NUM', '一般機車':'MOTOR_NUM'})
park_df3 = park_df3[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [332]:
park_df1.shape, park_df2.shape, park_df3.shape

((234, 6), (127, 6), (84, 6))

In [333]:
park_df = pd.concat([park_df1, park_df2, park_df3], axis=0).reset_index(drop=True)

In [335]:
park_df.to_csv('停車/臺南停車場/台南停車場(processed).csv', index=False, encoding='utf-8-sig')

## 高雄

In [336]:
import requests
import json
url = 'http://parkinglocation.tbkc.gov.tw/getjson.ashx'
results = requests.get(url)
json_result = json.loads(results.text)

In [340]:
NAME, ADDRESS, CAR_NUM, MOTOR_NUM, lat, lng = [],[],[],[],[],[]
for result in json_result:
    NAME.append(result['停車場名稱'])
    ADDRESS.append(result['停車場位置'])
    CAR_NUM.append(result['小型車車位'])
    MOTOR_NUM.append(result['機車車位'])
    lat.append(result['緯度'])
    lng.append(result['經度'])

In [343]:
df = pd.DataFrame({'NAME':NAME, 'ADDRESS':ADDRESS, 'CAR_NUM':CAR_NUM, 'MOTOR_NUM':MOTOR_NUM, 'lat':lat, 'lng':lng})
df['CAR_NUM'] = df['CAR_NUM'].fillna(0)
df['MOTOR_NUM'] = df['MOTOR_NUM'].fillna(0)

In [345]:
df.to_csv('停車/高雄停車場/高雄民營停車場(processed).csv', index=False, encoding='utf-8-sig')

In [347]:
park_df = pd.read_csv('停車/高雄停車場/高雄公有109.3.9.csv', encoding='utf-8')

In [351]:
park_df
park_df = park_df.rename(columns={'停車場名稱':'NAME', '停車場位置':'ADDRESS', '小型車':'CAR_NUM', '機車':'MOTOR_NUM', '座標-緯度':'lat', '座標-經度':'lng'})
park_df = park_df[['NAME', 'ADDRESS', 'CAR_NUM', 'MOTOR_NUM', 'lat', 'lng']]

In [353]:
park_df['CAR_NUM'] = park_df['CAR_NUM'].fillna(0)
park_df['MOTOR_NUM'] = park_df['MOTOR_NUM'].fillna(0)
park_df

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,財稅大樓,中正三路25號,126.0,0.0,22.630674,120.309858
1,忠孝,忠孝一路331號,568.0,0.0,22.633395,120.306056
2,民權輕鋼架,民權一路221號,256.0,203.0,22.623527,120.311551
3,錦田機車,中正三路、錦田路口,0.0,124.0,22.631477,120.310186
4,忠孝一路,中正三路與忠孝一路口旁,40.0,7.0,22.630912,120.3061048
...,...,...,...,...,...,...
239,五里埔第二基地,街底土地公廟旁（5席）,5.0,0.0,22.980929,120.554136
240,大愛A區,杉林國中斜對面,14.0,0.0,22.969360,120.542510
241,大愛B區,和氣街110巷叉口,40.0,0.0,22.966443,120.542328
242,大愛C區,合心路旁,66.0,0.0,22.962941,120.539572


In [355]:
park_df = park_df.dropna(subset=['lat', 'lng']).reset_index(drop=True)

In [356]:
park_df

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,財稅大樓,中正三路25號,126.0,0.0,22.630674,120.309858
1,忠孝,忠孝一路331號,568.0,0.0,22.633395,120.306056
2,民權輕鋼架,民權一路221號,256.0,203.0,22.623527,120.311551
3,錦田機車,中正三路、錦田路口,0.0,124.0,22.631477,120.310186
4,忠孝一路,中正三路與忠孝一路口旁,40.0,7.0,22.630912,120.3061048
...,...,...,...,...,...,...
236,五里埔第一基地,日光三街旁(12席),12.0,0.0,22.980929,120.554136
237,五里埔第二基地,街底土地公廟旁（5席）,5.0,0.0,22.980929,120.554136
238,大愛A區,杉林國中斜對面,14.0,0.0,22.969360,120.542510
239,大愛B區,和氣街110巷叉口,40.0,0.0,22.966443,120.542328


In [358]:
park_df.to_csv('停車/高雄停車場/高雄公有109.3.9(processed).csv', index=False, encoding='utf-8-sig')

In [359]:
park_df1 = pd.read_csv('停車/高雄停車場/高雄公有109.3.9(processed).csv', encoding='utf-8')
park_df2 = pd.read_csv('停車/高雄停車場/高雄民營停車場(processed).csv', encoding='utf-8')

In [360]:
park_df1

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,財稅大樓,中正三路25號,126.0,0.0,22.630674,120.309858
1,忠孝,忠孝一路331號,568.0,0.0,22.633395,120.306056
2,民權輕鋼架,民權一路221號,256.0,203.0,22.623527,120.311551
3,錦田機車,中正三路、錦田路口,0.0,124.0,22.631477,120.310186
4,忠孝一路,中正三路與忠孝一路口旁,40.0,7.0,22.630912,120.3061048
...,...,...,...,...,...,...
236,五里埔第一基地,日光三街旁(12席),12.0,0.0,22.980929,120.554136
237,五里埔第二基地,街底土地公廟旁（5席）,5.0,0.0,22.980929,120.554136
238,大愛A區,杉林國中斜對面,14.0,0.0,22.969360,120.542510
239,大愛B區,和氣街110巷叉口,40.0,0.0,22.966443,120.542328


In [361]:
park_df2

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,凹子底停車場,龍中段45地號土地（神農路、南屏路口）,192.0,137.0,22.657410,120.29950
1,英明停車場,林聖段一小段2017-4地號（英明路近英祥街口）,11.0,42.0,22.616890,120.32488
2,阮綜合醫院停車場,苓洲段1567地號（四維四路107號）(立體停車塔),208.0,0.0,22.614953,120.29796
3,紫龍城停車場,獅頭段一小段1689地號3層（陽明路169號）,50.0,0.0,22.640710,120.34138
4,財團法人天主教聖功醫院附設地下室停車場,正明段350、350-1地號1-3層（建國一路352號）,103.0,0.0,22.633780,120.32406
...,...,...,...,...,...,...
822,三塊厝自立停車場,三塊厝段四小段3022地號土地(自立一路與察哈爾二街口東南側),24.0,0.0,22.647960,120.29700
823,寶盛九如路停車場,新都段186地號土地(九如三路與中華二路157巷西北側),60.0,0.0,22.643000,120.29008
824,寶盛至聖博愛停車場,龍中段22地號土地(林皇宮百匯西側),44.0,0.0,22.660000,120.30197
825,大樂停車場,灣內段700地號(民族一路463號室外地面層)俥亭重文街停車場\t左營區\t新光段118地號...,84.0,0.0,22.683890,120.30669


In [365]:
park_df = pd.concat([park_df1, park_df2], axis=0).reset_index(drop=True)

In [367]:
park_df.to_csv('停車/高雄停車場/高雄停車場(processed).csv', index=False, encoding='utf-8-sig')

In [368]:
park_df

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,財稅大樓,中正三路25號,126.0,0.0,22.630674,120.309858
1,忠孝,忠孝一路331號,568.0,0.0,22.633395,120.306056
2,民權輕鋼架,民權一路221號,256.0,203.0,22.623527,120.311551
3,錦田機車,中正三路、錦田路口,0.0,124.0,22.631477,120.310186
4,忠孝一路,中正三路與忠孝一路口旁,40.0,7.0,22.630912,120.3061048
...,...,...,...,...,...,...
1063,三塊厝自立停車場,三塊厝段四小段3022地號土地(自立一路與察哈爾二街口東南側),24.0,0.0,22.647960,120.297
1064,寶盛九如路停車場,新都段186地號土地(九如三路與中華二路157巷西北側),60.0,0.0,22.643000,120.29
1065,寶盛至聖博愛停車場,龍中段22地號土地(林皇宮百匯西側),44.0,0.0,22.660000,120.302
1066,大樂停車場,灣內段700地號(民族一路463號室外地面層)俥亭重文街停車場\t左營區\t新光段118地號...,84.0,0.0,22.683890,120.307


In [369]:
park_df1 = pd.read_csv('停車/新北市停車場/新北停車場(processed).csv', encoding='utf-8')
park_df2 = pd.read_csv('停車/台北市停車場/台北停車場(processed).csv', encoding='utf-8')

In [370]:
park_df1

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,遠東百貨停車場,板橋區中山路一段152號,453,0,25.010925,121.464492
1,南華停車場,中和區中和路281號邊,42,0,24.999567,121.506097
2,集美國小地下停車場,三重區集美街10號,93,0,25.056067,121.488196
3,十三行博物館第二停車場,八里區博物館路200號,105,21,25.155692,121.405736
4,中和國中停車場,中和區興南路二段3巷17號,17,0,24.986143,121.508558
...,...,...,...,...,...,...
1067,三芝中正立體停車場,新北市三芝區中山路二段48-1號,261,34,25.254961,121.503378
1068,新北市萬板大橋停車場,萬板大橋板橋端至14號橋墩下方空間(不含道路、19號橋墩至20號橋墩間龍翠聯合活動中心、20...,389,179,25.027824,121.483630
1069,金城立體停車場,新北市土城區金城路2段266號,537,0,24.983801,121.458432
1070,貢寮澳底公園地下停車場,新北市貢寮區仁愛路43號B1,88,22,25.053768,121.925625


In [371]:
park_df2

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,臺北市立金華國民中學,新生南路2段32號,49,0,25.031408,121.533080
1,墾海停車場,忠孝東路1段23號B2、B3,66,0,25.044875,121.524818
2,TIMES新湖三路,舊宗段32、32-11地號；新湖三路與新湖三路219巷交叉口西南側空地,38,0,25.065239,121.580741
3,歐特儀股份有限公司冷水坑2號停車場,NaN,48,24,25.167975,121.563313
4,陽明山花鐘停車場,湖山路2段(陽明山公園花鐘前),45,0,25.158503,121.539274
...,...,...,...,...,...,...
1957,Times長安新生,長安段4小段525地號；長安東路二段36巷18號。,22,7,25.047450,121.531244
1958,嘟嘟房玉成站停車場,南港區玉成段5小段258、258-1、258-2、234-1、234-3地號；忠孝東路六段,49,0,25.046094,121.583854
1959,華陰街臨時平面停車場,華陰街18號對面\r\n,25,0,25.049232,121.520362
1960,國立臺灣科技大學國際大樓地下停車場,NaN,263,830,25.013511,121.540852


In [380]:
park_df = pd.concat([park_df1, park_df2],axis=0).drop_duplicates(subset=['NAME', 'CAR_NUM']).reset_index(drop=True)

In [381]:
park_df

,NAME,ADDRESS,CAR_NUM,MOTOR_NUM,lat,lng
0,遠東百貨停車場,板橋區中山路一段152號,453,0,25.010925,121.464492
1,南華停車場,中和區中和路281號邊,42,0,24.999567,121.506097
2,集美國小地下停車場,三重區集美街10號,93,0,25.056067,121.488196
3,十三行博物館第二停車場,八里區博物館路200號,105,21,25.155692,121.405736
4,中和國中停車場,中和區興南路二段3巷17號,17,0,24.986143,121.508558
...,...,...,...,...,...,...
2973,Times長安新生,長安段4小段525地號；長安東路二段36巷18號。,22,7,25.047450,121.531244
2974,嘟嘟房玉成站停車場,南港區玉成段5小段258、258-1、258-2、234-1、234-3地號；忠孝東路六段,49,0,25.046094,121.583854
2975,華陰街臨時平面停車場,華陰街18號對面\r\n,25,0,25.049232,121.520362
2976,國立臺灣科技大學國際大樓地下停車場,NaN,263,830,25.013511,121.540852


In [383]:
park_df.to_csv('停車/雙北停車場(processed).csv', index=False, encoding='utf-8-sig')